<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="헤더" style="width: 400px;"/> </a>

# 4.0 모델 사용

이 노트북에서는 서버를 "사용"하고 질문-답변 NLP 작업에서 실시간으로 작동되는 추론을 살펴봅니다.

**[4.1 API의 기초](#4.1-API의-기초)**<br>
**[4.2 추론 API 개요](#4.2-추론-API-개요)**<br>
**[4.3 요청 준비](#4.3-요청-준비)**<br>
**[4.4 서버 쿼리](#4.4-서버-쿼리)**<br>
**[4.5 응답 사후 처리](#4.5-응답-사후-처리)**<br>

Triton Inference Server는 HTTP 및 gRPC 엔드포인트를 사용하여 서비스를 노출합니다. 따라서 다양한 도구(예: Java, C++, C, Python, PHP, Ruby 등에서 사용할 수 있는 [gRPC](https://grpc.io/docs/languages/))를# 사용하여 서버를 쿼리할 수 있습니다.  Triton은 자체 서빙 표준을 구현하지 않으며, 그 대신 [KFServing Predict 프로토콜 버전 2](https://github.com/kubeflow/kfserving/tree/master/docs/predict-api/v2)를 사용하여 서비스를 노출합니다. 따라서 서빙 서비스를 구현하는 다양한 도구와 호환됩니다.  

향후 개발을 간소화하기 위해 Triton은 다음과 같은 여러 API를 통해 서버 프로토콜을 노출합니다.
- [Python API](https://docs.nvidia.com/deeplearning/triton-inference-server/master-user-guide/docs/python_api.html?highlight=grpc)
- [C++ API](https://docs.nvidia.com/deeplearning/triton-inference-server/master-user-guide/docs/cpp_api/cpp_api_root.html)
- [Protobuf API](https://docs.nvidia.com/deeplearning/triton-inference-server/master-user-guide/docs/protobuf_api/protobuf_api_root.html)

이 예에서는 Python API를 사용하여 질문-답변 서비스를 사용하는 방법을 배웁니다.

# 4.1 API의 기초

먼저 API의 기본 구성 요소를 검토해보겠습니다. 핵심 엘리먼트는 <a href="https://ngc.nvidia.com/catalog/containers/nvidia:tritonserver">Triton 클라이언트 유틸리티</a>를 사용하여 NGC 컨테이너를 다운로드하거나 <a href="https://github.com/NVIDIA/triton-inference-server/releases/tag/v2.0.0">Triton GitHub 리포지토리</a>에서 직접 다운로드하여 얻을 수 있는 <code>tritonhttpclient</code>입니다. 배포되었으면(이 수업에서는 이미 설치되어 있음) 해당 라이브러리를 가져옵니다.

In [ ]:
import os
import json
import argparse
import numpy as np
import tritonhttpclient

첫 번째 단계는 클라이언트가 서버를 가리키게 해서 클라이언트를 초기화하는 것입니다.

In [ ]:
try:
    triton_client = tritonhttpclient.InferenceServerClient(url="triton:8000", verbose=True)
except Exception as e:
    print("channel creation failed: " + str(e))

그런 다음, 서버의 상태와 모델의 가용성 및 상태를 검사합니다.

In [ ]:
modelName = "bertQA-torchscript"
print(triton_client.is_server_live())
print(triton_client.is_server_ready())
print(triton_client.is_model_ready(modelName,"1"))

마지막으로, 서버가 반환한 메타데이터를 검사합니다.

In [ ]:
triton_client.get_server_metadata()

기본 상태 확인 및 모델 추론 외에도 API는 서버에 대한 세분화된 제어를 제공하여 모델 로드 및 언로드와 같은 작업을 가능하게 합니다. 자세한 내용은 <a href="https://docs.nvidia.com/deeplearning/triton-inference-server/master-user-guide/docs/python_api.html">문서</a> 및 <a href="https://github.com/NVIDIA/triton-inference-server/tree/60c33d5593ad0d50716f04f69bb4b24ee3a7996d/src/clients/python/examples">API 예</a>를 참조하십시오.

# 4.2 추론 API 개요

질문 답변을 수행하기 위해 구축된 뉴럴 네트워크를 사용하고 있으므로 서버에 대한 예제 쿼리를 실행하겠습니다. 먼저, 서버가 사용할 입력 및 출력 데이터의 형태를 살펴보겠습니다.

In [ ]:
triton_client.get_model_metadata(modelName)

아래와 비슷한 응답을 수신해야 합니다. <br/>
<img width=1000 src="images/DataFormat.png"/>

서버가 다음 3개의 입력 텐서를 요구하고 있음을 나타냈습니다.
- input__0 being the input_ids
- input_1 being the sequence_ids
- input_2 being the mask_ids

서버가 다음과 같이 응답합니다.
- output__0 being the start logits
- output_-1 being the end logits

이제 질문과 맥락을 서버에 필요한 형식으로 사전 처리해야 합니다.

# 4.3 요청 준비

먼저 질문과 답변을 만듭니다.

In [ ]:
question = "Most antibiotics target bacteria and don't affect what class of organisms? "
context = "Within the genitourinary and gastrointestinal tracts, commensal flora serve as biological barriers by " +\
        "competing with pathogenic bacteria for food and space and, in some cases, by changing the conditions in " +\
        "their environment, such as pH or available iron. This reduces the probability that pathogens will " +\
        "reach sufficient numbers to cause illness. However, since most antibiotics non-specifically target bacteria" +\
        "and do not affect fungi, oral antibiotics can lead to an overgrowth of fungi and cause conditions such as a" +\
        "vaginal candidiasis (a yeast infection). There is good evidence that re-introduction of probiotic flora, such " +\
        "as pure cultures of the lactobacilli normally found in unpasteurized yogurt, helps restore a healthy balance of" +\
        "microbial populations in intestinal infections in children and encouraging preliminary data in studies on bacterial " +\
        "gastroenteritis, inflammatory bowel diseases, urinary tract infection and post-surgical infections. " 

두 번째로, 추가 유틸리티를 가져와서 데이터 변환에 필요한 상용구 로직을 숨깁니다.

In [ ]:
import sys
sys.path.insert(0,'/dli/task/client')
from tokenization import BertTokenizer
from inference import preprocess_tokenized_text,parse_answer

이 코드 섹션은 데이터를 필요한 형식으로 변환합니다.

In [ ]:
tokenizer = BertTokenizer("/dli/task/vocab", do_lower_case=True, max_len=512) 
doc_tokens = context.split()
query_tokens = tokenizer.tokenize(question)

tensors_for_inference, tokens_for_postprocessing = preprocess_tokenized_text(doc_tokens, 
                                    query_tokens, 
                                    tokenizer, 
                                    max_seq_length=384, 
                                    max_query_length=64)

dtype = np.int64
input_ids = np.array(tensors_for_inference.input_ids, dtype=dtype)[None,...] # make bs=1
segment_ids = np.array(tensors_for_inference.segment_ids, dtype=dtype)[None,...] # make bs=1
input_mask = np.array(tensors_for_inference.input_mask, dtype=dtype)[None,...] # make bs=1

마지막으로, 데이터를 Triton에 필요한 구조로 복사합니다. 앞서 Triton 서버 응답에서 지정한 텐서 이름, 데이터 유형 및 텐서 차원을 사용한다는 점에 주목하십시오.

In [ ]:
inputs = []
inputs.append(tritonhttpclient.InferInput('input__0', [1, len(input_ids[0])], "INT64"))
inputs.append(tritonhttpclient.InferInput('input__1', [1, len(segment_ids[0])], "INT64"))
inputs.append(tritonhttpclient.InferInput('input__2', [1, len(input_mask[0])], "INT64"))


inputs[0].set_data_from_numpy(input_ids, binary_data=False)
inputs[1].set_data_from_numpy(segment_ids, binary_data=False)
inputs[2].set_data_from_numpy(input_mask, binary_data=False)

이러한 입력 중 하나를 검사하여 네트워크에 필요한 숫자 형식으로 변환되고 토큰화된 새로운 데이터 표현을 찾을 수 있습니다.

In [ ]:
inputs[0]._get_tensor()

요청과 관련된 모든 출력 텐서를 그냥 가져올 수도 있지만 최소한의 텐서만 가져와서 대역폭을 최소화하는 것이 좋습니다. 이렇게 하려면 요청 출력을 지정합니다.

In [ ]:
outputs = []
outputs.append(
        tritonhttpclient.InferRequestedOutput('output__0', binary_data=False))
outputs.append(
        tritonhttpclient.InferRequestedOutput('output__1', binary_data=False))

# 4.4 서버 쿼리

이제 서버에 요청을 발행해 보겠습니다. <code>출력</code> 매개변수는 옵션입니다. 지정되지 않은 경우 모든 텐서가 반환됩니다.

In [ ]:
results = triton_client.infer(modelName,
                                  inputs,
                                  outputs=outputs)

보시다시피 <code>결과</code>와 <code>출력</code>은 동일한 데이터 유형입니다.  

In [ ]:
results
outputs

# 4.5 응답 사후 처리

우리 사례에서의 결과는 그저 시작 및 끝 위치의 로짓입니다. 그러한 로짓을 추가로 처리하여 인간이 읽을 수 있는 결과를 만들어 보겠습니다. 먼저 추가 처리를 보다 쉽게 수행할 수 있도록 벡터를 NumPy에 복사합니다.

In [ ]:
# Validate the results by comparing with precomputed values.
output0_data = results.as_numpy('output__0')
output1_data = results.as_numpy('output__1')

결과를 살펴보고...

In [ ]:
output0_data

...인간이 읽을 수 있는 형식으로 변환해 보겠습니다.

In [ ]:
start_logits = output0_data[0].tolist()
end_logits = output1_data[0].tolist()

answer, answers = parse_answer(doc_tokens, tokens_for_postprocessing, 
                                 start_logits, end_logits)

# print result
print()
print(answer)
print()
print(json.dumps(answers, indent=4))

그리고 ... 자, 됐습니다!  결과를 얻었습니다! 언제든지 본인의 쿼리로 실험해 보십시오.

<h3 style="color:green;">축하합니다!</h3><br>
과정을 완료했습니다! 

인증서를 받으려면 과정 설문조사를 작성하고 학생 평가를 수행해 주십시오.

<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="헤더" style="width: 400px;"/> </a>